Прогнозирование Trans-MAF нерегулярных временных рядов + фичи на основе даты.

## Imports

In [1]:
import copy
import json
import os
import pickle
import sys
import warnings

sys.path.append('../pytorch-ts')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns
import torch
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from gluonts.dataset.split import split
from gluonts.dataset.util import to_pandas
from gluonts.evaluation import MultivariateEvaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from pts import Trainer
from pts.model.transformer_tempflow import TransformerTempFlowEstimator

/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_38812/359591521.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/artificial/_base.py:84: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  self.freq: BaseOffset = t

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Read Datasets

In [3]:
exchange = get_dataset("exchange_rate", regenerate=False)

solar = get_dataset("solar_nips", regenerate=False)

electricity = get_dataset("electricity_nips", regenerate=False)

In [4]:
# electricity
electricity_train_grouper = MultivariateGrouper(
    max_target_dim=min(2000, int(electricity.metadata.feat_static_cat[0].cardinality))
    )

electricity_test_grouper = MultivariateGrouper(
    num_test_dates=int(len(electricity.test) / len(electricity.train)), 
    max_target_dim=min(2000, int(electricity.metadata.feat_static_cat[0].cardinality))
    )

# solar
solar_train_grouper = MultivariateGrouper(
    max_target_dim=min(2000, int(solar.metadata.feat_static_cat[0].cardinality))
    )

solar_test_grouper = MultivariateGrouper(
    num_test_dates=int(len(solar.test) / len(solar.train)), 
    max_target_dim=min(2000, int(solar.metadata.feat_static_cat[0].cardinality))
    )

# exchange
exchange_train_grouper = MultivariateGrouper(
    max_target_dim=min(2000, int(exchange.metadata.feat_static_cat[0].cardinality))
    )

exchange_test_grouper = MultivariateGrouper(
    num_test_dates=int(len(exchange.test) / len(exchange.train)), 
    max_target_dim=min(2000, int(exchange.metadata.feat_static_cat[0].cardinality))
    )

In [5]:
# solar
solar_dataset_train = solar_train_grouper(solar.train)
solar_dataset_test = solar_test_grouper(solar.test)

# electricity
electricity_dataset_train = electricity_train_grouper(electricity.train)
electricity_dataset_test = electricity_test_grouper(electricity.test)

# exchange
exchange_dataset_train = exchange_train_grouper(exchange.train)
exchange_dataset_test = exchange_test_grouper(exchange.test)

/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/common.py:262: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  return pd.Period(val, freq)
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/common.py:262: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return pd.Period(val, freq)
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/multivariate_grouper.py:113: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  timestamp + len(data[FieldName.TARGET]) - 1,
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/multivariate_grouper.py:242: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex

In [6]:
def transform_cyclical_features(
    data: pd.DataFrame, 
    date_col: str, 
    agg_level: str
) -> pd.DataFrame:
    """
    Функция создает фичи на основе даты и производит sin/cos преобразование циклических признаков.
            - xsin = SIN((2 * pi * x) / max(x))
            - xcos = COS((2 * pi * x) / max(x))

    :param agg_level: str уровень аггрегации: [D, W, M]

    """
    available_features = ["weekday", "day", "dayofyear", "week", "month", "hour"]

    n_unique_values_map = {
        "weekday": 7,
        "day": 31,
        "dayofyear": 365,
        "week": 52,
        "month": 12,
        "hour": 24
    }

    
    if agg_level == 'D' or agg_level == 'B':
        data["day"] = data[date_col].dt.day
        data["dayofyear"] = data[date_col].dt.dayofyear
        data["weekday"] = data[date_col].dt.dayofweek
        data["week"] = data[date_col].dt.isocalendar().week
        data["month"] = data[date_col].dt.month

    elif agg_level == 'H':
        data["day"] = data[date_col].dt.day
        data["weekday"] = data[date_col].dt.dayofweek
        data["hour"] = data[date_col].dt.hour

    cyclical_features = list(set(data.columns) & set(available_features))

    for feature in cyclical_features:
        data[feature] = data[feature].astype(int)

        data[f"{feature}_sin"] = np.sin(
            (data[feature] * 2 * np.pi) / n_unique_values_map[feature]
        )
        data[f"{feature}_cos"] = np.cos(
            (data[feature] * 2 * np.pi) / n_unique_values_map[feature]
        )

    data = data.drop(cyclical_features + [date_col], axis=1)

    return np.array(data.T)

def create_date_features(data, raw_dataset, agg_level):
    """"""
    all_dates = pd.date_range(
        start=data['start'].to_timestamp(), 
        periods=data['target'].shape[1],
        freq=raw_dataset.metadata.freq
    )

    date_df = pd.DataFrame({
        "date": all_dates
    })

    date_features = transform_cyclical_features(
        data=date_df,
        date_col='date',
        agg_level=agg_level
    )

    data['time_feat'] = date_features

    return data

## Train models

In [7]:
RUN_FOLDER = './reports/maf/run_03'

MODELS_FOLDER = os.path.join(RUN_FOLDER, 'models')

PREDICTIONS_FOLDER = os.path.join(RUN_FOLDER, 'predictions')

for folder in [RUN_FOLDER, MODELS_FOLDER, PREDICTIONS_FOLDER]:
    if not os.path.exists(folder):
        os.mkdir(folder)

### Solar

#### Train

```number of features x target length```

In [75]:
# time features for train dataset

solar_dataset_train[0] = create_date_features(
    data=solar_dataset_train[0], 
    raw_dataset=solar, 
    agg_level='H'
)

assert solar_dataset_train[0]['time_feat'].shape[1] == solar_dataset_train[0]['target'].shape[1]

/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_63947/24479359.py:56: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_dates = pd.date_range(


In [77]:
# time features for test dataset

for i in range(len(solar_dataset_test)):
    solar_dataset_test[i] = create_date_features(
        data=solar_dataset_test[i], 
        raw_dataset=solar, 
        agg_level='H'
    )

    assert solar_dataset_test[i]['time_feat'].shape[1] == solar_dataset_test[i]['target'].shape[1]

/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_63947/24479359.py:56: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_dates = pd.date_range(
/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_63947/24479359.py:56: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_dates = pd.date_range(
/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_63947/24479359.py:56: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_dates = pd.date_range(
/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_63947/24479359.py:56: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_dates = pd.date_range(
/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_63947/24479359.py:56: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h

In [97]:
%%time

train_size = solar_dataset_train[0]['target'].shape[1]
n_series = solar_dataset_train[0]['target'].shape[0]

for i in np.linspace(0, 0.8, 9):
    
    print(f"\n### NaNs probability = {i} ###\n")
    
    train = copy.copy(solar_dataset_train)
    test = copy.copy(solar_dataset_test)

    nans_prob = np.round(i, 1)

    for j in range(n_series):

        # take random indices
        idxs = np.random.choice(range(0, train_size), size=int(nans_prob * train_size), replace=False)

        # nan train values
        train[0]['target'][j, idxs] = np.nan

        # nan test values
        for k in range(len(test)):
            test[k]['target'][j, idxs] = np.nan
    
    train = list(train) * 100

    # train
    estimator = TransformerTempFlowEstimator(
        input_size=278,
        target_dim=int(solar.metadata.feat_static_cat[0].cardinality),
        prediction_length=solar.metadata.prediction_length,
        context_length=solar.metadata.prediction_length * 4,
        flow_type='MAF',
        dequantize=True,
        freq=solar.metadata.freq,
        trainer=Trainer(
            device='cpu',
            epochs=14,
            learning_rate=1e-3,
            num_batches_per_epoch=100,
            batch_size=64,
        )
    )

    predictor = estimator.train(train, num_workers=4)

    # save model
    torch.save(predictor, os.path.join(MODELS_FOLDER, f"estimator__solar_{nans_prob}.ckpt"))

    solar_dataset_train = solar_train_grouper(solar.train)
    solar_dataset_test = solar_test_grouper(solar.test)


### NaNs probability = 0.0 ###



/Users/eliseev_s/Documents/Projects/00_GenModels/trans_maf/notebooks/../pytorch-ts/pts/feature/lags.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = to_offset(freq_str)
/Users/eliseev_s/Documents/Projects/00_GenModels/trans_maf/notebooks/../pytorch-ts/pts/feature/fourier_date_feature.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = to_offset(freq_str)
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
  0%|          | 0/99 [00:00<?, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages


### NaNs probability = 0.1 ###



  0%|          | 0/99 [00:00<?, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserial


### NaNs probability = 0.2 ###



...


### NaNs probability = 0.7000000000000001 ###



  0%|          | 0/99 [00:00<?, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserial


### NaNs probability = 0.8 ###



  0%|          | 0/99 [00:00<?, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserial

CPU times: user 1h 26min 17s, sys: 35min 3s, total: 2h 1min 20s
Wall time: 2h 30min 43s


#### Predict

In [98]:
n_series = solar_dataset_train[0]['target'].shape[0]
train_size = solar_dataset_train[0]['target'].shape[1]

for i in np.linspace(0, 0.8, 9):
    print(f"\n### NaNs probability = {i} ###\n")

    nans_prob = np.round(i, 1)

    predictor = torch.load(os.path.join(MODELS_FOLDER, f"estimator__solar_{nans_prob}.ckpt"))

    test = copy.copy(solar_dataset_test)
    
    for j in range(n_series):

        # take random indices
        idxs = np.random.choice(range(0, train_size), size=int(nans_prob * train_size), replace=False)

        # nan test values
        for k in range(len(test)):
            test[k]['target'][j, idxs] = np.nan

    success = 0
    num_samples = 20

    while success != 1 and num_samples != 100:
        try:
            # evaluate model
            evaluator = MultivariateEvaluator(
                quantiles=(np.arange(20)/20.0)[1:],
                target_agg_funcs={'sum': np.sum}
            )

            forecast_it, ts_it = make_evaluation_predictions(
                dataset=test,
                predictor=predictor,
                num_samples=num_samples
            )

            forecasts = list(forecast_it)
            targets = list(ts_it)

            agg_metric, _ = evaluator(targets, forecasts, num_series=len(test))

            # save metrics
            with open(os.path.join(PREDICTIONS_FOLDER, f"agg_metrics__solar_{nans_prob}.json"), 'w') as fp:
                json.dump(agg_metric, fp)
            
            # save forecasts
            with open(os.path.join(PREDICTIONS_FOLDER, f"targets__solar_{nans_prob}.json"), 'wb') as handle:
                pickle.dump(targets, handle, protocol=pickle.HIGHEST_PROTOCOL)
                
            with open(os.path.join(PREDICTIONS_FOLDER, f"forecasts__solar_{nans_prob}.json"), 'wb') as handle:
                pickle.dump(forecasts, handle, protocol=pickle.HIGHEST_PROTOCOL)

            success = 1

        except:
            num_samples += 1

    if num_samples == 100 and success == 0:
        print(print(f"\n### NaNs probability = {i} NOT SUCCESS ###\n"))
        
    solar_dataset_test = solar_test_grouper(solar.test)


### NaNs probability = 0.0 ###



Running evaluation: 7it [00:00, 227.51it/s]
Running evaluation: 7it [00:00, 269.78it/s]
Running evaluation: 7it [00:00, 243.32it/s]
Running evaluation: 7it [00:00, 259.68it/s]
Running evaluation: 7it [00:00, 253.23it/s]
Running evaluation: 7it [00:00, 234.89it/s]
Running evaluation: 7it [00:00, 259.44it/s]
Running evaluation: 7it [00:00, 259.20it/s]
Running evaluation: 7it [00:00, 234.54it/s]
Running evaluation: 7it [00:00, 266.82it/s]
Running evaluation: 7it [00:00, 259.29it/s]
Running evaluation: 7it [00:00, 242.61it/s]
Running evaluation: 7it [00:00, 256.50it/s]
Running evaluation: 7it [00:00, 255.35it/s]
Running evaluation: 7it [00:00, 251.49it/s]
Running evaluation: 7it [00:00, 265.68it/s]
Running evaluation: 7it [00:00, 270.47it/s]
Running evaluation: 7it [00:00, 237.93it/s]
Running evaluation: 7it [00:00, 259.81it/s]
Running evaluation: 7it [00:00, 261.17it/s]
Running evaluation: 7it [00:00, 216.20it/s]
Running evaluation: 7it [00:00, 255.17it/s]
Running evaluation: 7it [00:00, 


### NaNs probability = 0.1 ###



Running evaluation: 7it [00:00, 257.40it/s]
Running evaluation: 7it [00:00, 218.39it/s]
Running evaluation: 7it [00:00, 253.42it/s]
Running evaluation: 7it [00:00, 242.20it/s]
Running evaluation: 7it [00:00, 229.87it/s]
Running evaluation: 7it [00:00, 246.37it/s]
Running evaluation: 7it [00:00, 244.24it/s]
Running evaluation: 7it [00:00, 245.75it/s]
Running evaluation: 7it [00:00, 247.68it/s]
Running evaluation: 7it [00:00, 247.33it/s]
Running evaluation: 7it [00:00, 243.85it/s]
Running evaluation: 7it [00:00, 236.10it/s]
Running evaluation: 7it [00:00, 257.27it/s]
Running evaluation: 7it [00:00, 250.08it/s]
Running evaluation: 7it [00:00, 253.48it/s]
Running evaluation: 7it [00:00, 249.50it/s]
Running evaluation: 7it [00:00, 252.48it/s]
Running evaluation: 7it [00:00, 261.03it/s]
Running evaluation: 7it [00:00, 255.37it/s]
Running evaluation: 7it [00:00, 259.91it/s]
Running evaluation: 7it [00:00, 253.48it/s]
Running evaluation: 7it [00:00, 251.54it/s]
Running evaluation: 7it [00:00, 


### NaNs probability = 0.2 ###



...


### NaNs probability = 0.7000000000000001 ###



Running evaluation: 1it [00:00, 198.14it/s]
Running evaluation: 7it [00:00, 251.78it/s]
Running evaluation: 7it [00:00, 256.49it/s]
Running evaluation: 7it [00:00, 242.80it/s]
Running evaluation: 7it [00:00, 260.10it/s]
Running evaluation: 7it [00:00, 241.39it/s]
Running evaluation: 7it [00:00, 244.92it/s]
Running evaluation: 7it [00:00, 253.52it/s]
Running evaluation: 7it [00:00, 247.73it/s]
Running evaluation: 7it [00:00, 239.37it/s]
Running evaluation: 7it [00:00, 247.88it/s]
Running evaluation: 7it [00:00, 256.24it/s]
Running evaluation: 7it [00:00, 252.69it/s]
Running evaluation: 7it [00:00, 247.90it/s]
Running evaluation: 7it [00:00, 231.41it/s]
Running evaluation: 7it [00:00, 236.65it/s]
Running evaluation: 7it [00:00, 239.91it/s]
Running evaluation: 7it [00:00, 218.49it/s]
Running evaluation: 7it [00:00, 243.93it/s]
Running evaluation: 7it [00:00, 248.44it/s]
Running evaluation: 7it [00:00, 247.05it/s]
Running evaluation: 7it [00:00, 243.48it/s]
Running evaluation: 7it [00:00, 


### NaNs probability = 0.8 ###



Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [


### NaNs probability = 0.8 NOT SUCCESS ###

None


### Electricity

In [16]:
# time features for train dataset

electricity_dataset_train[0] = create_date_features(
    data=electricity_dataset_train[0], 
    raw_dataset=electricity, 
    agg_level='H'
)

assert electricity_dataset_train[0]['time_feat'].shape[1] == electricity_dataset_train[0]['target'].shape[1]

/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_38812/1119446166.py:56: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_dates = pd.date_range(


In [17]:
# time features for test dataset

for i in range(len(electricity_dataset_test)):
    electricity_dataset_test[i] = create_date_features(
        data=electricity_dataset_test[i], 
        raw_dataset=electricity, 
        agg_level='H'
    )

    assert electricity_dataset_test[i]['time_feat'].shape[1] == electricity_dataset_test[i]['target'].shape[1]

/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_38812/1119446166.py:56: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_dates = pd.date_range(
/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_38812/1119446166.py:56: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_dates = pd.date_range(
/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_38812/1119446166.py:56: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_dates = pd.date_range(
/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_38812/1119446166.py:56: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  all_dates = pd.date_range(
/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_38812/1119446166.py:56: FutureWarning: 'H' is deprecated and will be removed in a future version, ple

#### Train

In [11]:
%%time

train_size = electricity_dataset_train[0]['target'].shape[1]
test_size = electricity_dataset_test[0]['target'].shape[1]

n_series = electricity_dataset_train[0]['target'].shape[0]

for i in np.linspace(0.5, 0.8, 4):
    print(f"\n### NaNs probability = {i} ###\n")
    
    train = copy.copy(electricity_dataset_train)
    test = copy.copy(electricity_dataset_test)

    nans_prob = np.round(i, 1)

    for j in range(n_series):

        # take random indices
        idxs = np.random.choice(range(0, train_size), size=int(nans_prob * train_size), replace=False)

        # nan train values
        train[0]['target'][j, idxs] = np.nan

        # nan test values
        for k in range(len(test)):
            idxs = np.random.choice(range(0, test_size), size=int(nans_prob * test_size), replace=False)

            test[k]['target'][j, idxs] = np.nan
    
    train = list(train) * 100

    # train
    estimator = TransformerTempFlowEstimator(
        input_size=744,
        target_dim=int(electricity.metadata.feat_static_cat[0].cardinality),
        prediction_length=electricity.metadata.prediction_length,
        context_length=electricity.metadata.prediction_length * 4,
        flow_type='MAF',
        dequantize=True,
        freq=electricity.metadata.freq,
        trainer=Trainer(
            device='cpu',
            epochs=14,
            learning_rate=1e-3,
            num_batches_per_epoch=100,
            batch_size=64,
        )
    )

    predictor = estimator.train(train, num_workers=4)

    # save model
    torch.save(predictor, os.path.join(MODELS_FOLDER, f"estimator__electricity_{nans_prob}.ckpt"))

    electricity_dataset_train = electricity_train_grouper(electricity.train)
    electricity_dataset_test = electricity_test_grouper(electricity.test)


### NaNs probability = 0.5 ###



/Users/eliseev_s/Documents/Projects/00_GenModels/trans_maf/notebooks/../pytorch-ts/pts/feature/lags.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = to_offset(freq_str)
/Users/eliseev_s/Documents/Projects/00_GenModels/trans_maf/notebooks/../pytorch-ts/pts/feature/fourier_date_feature.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = to_offset(freq_str)
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
  0%|          | 0/99 [00:00<?, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages


### NaNs probability = 0.6 ###



/Users/eliseev_s/Documents/Projects/00_GenModels/trans_maf/notebooks/../pytorch-ts/pts/feature/lags.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = to_offset(freq_str)
/Users/eliseev_s/Documents/Projects/00_GenModels/trans_maf/notebooks/../pytorch-ts/pts/feature/fourier_date_feature.py:52: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  offset = to_offset(freq_str)
  0%|          | 0/99 [00:00<?, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization


### NaNs probability = 0.7000000000000001 ###



  0%|          | 0/99 [00:00<?, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserial


### NaNs probability = 0.8 ###



  0%|          | 0/99 [00:00<?, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserial

CPU times: user 47min 2s, sys: 20min 53s, total: 1h 7min 55s
Wall time: 1h 19min 33s


#### Predict

In [18]:
# in electricity dataset test size is less than train size 
# so we will take test size - prediction length indeces range to nan values

train_size = electricity_dataset_train[0]['target'].shape[1]
test_size = electricity_dataset_test[0]['target'].shape[1]
test_size = test_size - electricity.metadata.prediction_length

n_series = electricity_dataset_train[0]['target'].shape[0]

# for i in np.linspace(0.7, 0.8, 2):
for i in [0.1, 0.7, 0.8]:
    print(f"\n### NaNs probability = {i} ###\n")

    nans_prob = np.round(i, 1)

    predictor = torch.load(os.path.join(MODELS_FOLDER, f"estimator__electricity_{nans_prob}.ckpt"))

    test = copy.copy(electricity_dataset_test)
    
    for j in range(n_series):

        # nan test values
        for k in range(len(test)):
            idxs = np.random.choice(range(0, test_size), size=int(nans_prob * test_size), replace=False)

            test[k]['target'][j, idxs] = np.nan
    
    success = 0
    num_samples = 8

    while success != 1 and num_samples != 20:
        try:
            # evaluate model
            evaluator = MultivariateEvaluator(
                quantiles=(np.arange(20)/20.0)[1:],
                target_agg_funcs={'sum': np.sum}
            )

            forecast_it, ts_it = make_evaluation_predictions(
                dataset=test,
                predictor=predictor,
                num_samples=num_samples
            )

            forecasts = list(forecast_it)
            targets = list(ts_it)

            agg_metric, _ = evaluator(targets, forecasts, num_series=len(test))

            # save metrics
            with open(os.path.join(PREDICTIONS_FOLDER, f"agg_metrics__electricity_{nans_prob}.json"), 'w') as fp:
                json.dump(agg_metric, fp)
            
            # save forecasts
            with open(os.path.join(PREDICTIONS_FOLDER, f"targets__electricity_{nans_prob}.json"), 'wb') as handle:
                pickle.dump(targets, handle, protocol=pickle.HIGHEST_PROTOCOL)
                
            with open(os.path.join(PREDICTIONS_FOLDER, f"forecasts__electricity_{nans_prob}.json"), 'wb') as handle:
                pickle.dump(forecasts, handle, protocol=pickle.HIGHEST_PROTOCOL)

            success = 1

        except:
            num_samples += 1

    if num_samples == 100 and success == 0:
        print(print(f"\n### NaNs probability = {i} NOT SUCCESS ###\n"))
        
    electricity_dataset_test = electricity_test_grouper(electricity.test)


### NaNs probability = 0.1 ###



Running evaluation: 1it [00:00, 196.04it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]



### NaNs probability = 0.7 ###



Running evaluation: 1it [00:00, 207.20it/s]
Running evaluation: 1it [00:00, 219.68it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 1it [00:00, 218.35it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]



### NaNs probability = 0.8 ###



Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]
Running evaluation: 0it [00:00, ?it/s]


### Echange

In [12]:
exchange_freq = 'B'

In [13]:
# time features for train dataset

exchange_dataset_train[0] = create_date_features(
    data=exchange_dataset_train[0], 
    raw_dataset=exchange, 
    agg_level='B'
)

assert exchange_dataset_train[0]['time_feat'].shape[1] == exchange_dataset_train[0]['target'].shape[1]

# time features for test dataset

for i in range(len(exchange_dataset_test)):
    exchange_dataset_test[i] = create_date_features(
        data=exchange_dataset_test[i], 
        raw_dataset=exchange, 
        agg_level='B'
    )

    assert exchange_dataset_test[i]['time_feat'].shape[1] == exchange_dataset_test[i]['target'].shape[1]

/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_38812/1119446166.py:57: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  start=data['start'].to_timestamp(),
/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_38812/1119446166.py:57: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  start=data['start'].to_timestamp(),
/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_38812/1119446166.py:57: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  start=data['start'].to_timestamp(),
/var/folders/9z/qk2z_2yx52x3vny8hmx1wj2c0000gp/T/ipykernel_38812/1119446166.py:57: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  start=da

#### Train

In [14]:
%%time

train_size = exchange_dataset_train[0]['target'].shape[1]
n_series = exchange_dataset_train[0]['target'].shape[0]

for i in np.linspace(0, 0.8, 9):
    print(f"\n### NaNs probability = {i} ###\n")
    
    train = copy.copy(exchange_dataset_train)

    nans_prob = np.round(i, 1)

    for j in range(n_series):

        # take random indices
        idxs = np.random.choice(range(0, train_size), size=int(nans_prob * train_size), replace=False)

        # nan train values
        train[0]['target'][j, idxs] = np.nan
    
    train = list(train) * 100

    # train
    estimator = TransformerTempFlowEstimator(
        input_size=28,
        target_dim=int(exchange.metadata.feat_static_cat[0].cardinality),
        prediction_length=exchange.metadata.prediction_length,
        context_length=exchange.metadata.prediction_length * 8,
        flow_type='MAF',
        dequantize=True,
        freq=exchange_freq,
        scaling=False,
        trainer=Trainer(
            device='cpu',
            epochs=14, 
            learning_rate=1e-3,
            num_batches_per_epoch=100,
            batch_size=64,
        )
    )

    predictor = estimator.train(train, num_workers=4)

    # save model
    torch.save(predictor, os.path.join(MODELS_FOLDER, f"estimator__exchange_{nans_prob}.ckpt"))

    exchange_dataset_train = exchange_train_grouper(exchange.train)

/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")



### NaNs probability = 0.0 ###



  0%|          | 0/99 [00:00<?, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/multiprocessing/spawn.py:126: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  self = reduction.pickle.load(from_parent)
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/multiprocessing/spawn.py:126: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq


### NaNs probability = 0.1 ###



  0%|          | 0/99 [00:00<?, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/multiprocessing/spawn.py:126: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  self = reduction.pickle.load(from_parent)
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/multiprocessing/spawn.py:126: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq

...


### NaNs probability = 0.7000000000000001 ###



  0%|          | 0/99 [00:00<?, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/multiprocessing/spawn.py:126: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  self = reduction.pickle.load(from_parent)
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/multiprocessing/spawn.py:126: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq


### NaNs probability = 0.8 ###



  0%|          | 0/99 [00:00<?, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/multiprocessing/spawn.py:126: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  self = reduction.pickle.load(from_parent)
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/multiprocessing/spawn.py:126: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq

CPU times: user 4h 48min 2s, sys: 1h 20min 14s, total: 6h 8min 16s
Wall time: 4h 56min 21s


#### Predict

In [15]:
n_series = exchange_dataset_train[0]['target'].shape[0]
train_size = exchange_dataset_train[0]['target'].shape[1]

for i in np.linspace(0, 0.8, 9):
    print(f"\n### NaNs probability = {i} ###\n")

    nans_prob = np.round(i, 1)

    predictor = torch.load(os.path.join(MODELS_FOLDER, f"estimator__exchange_{nans_prob}.ckpt"))

    test = copy.copy(exchange_dataset_test)
    
    for j in range(n_series):

        # take random indices
        idxs = np.random.choice(range(0, train_size), size=int(nans_prob * train_size), replace=False)

        # nan test values
        for k in range(len(test)):
            test[k]['target'][j, idxs] = np.nan
    
    # evaluate model
    evaluator = MultivariateEvaluator(
        quantiles=(np.arange(20)/20.0)[1:],
        target_agg_funcs={'sum': np.sum}
    )

    # for num_preds in range(10, 101)[::-1]:
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=test,
        predictor=predictor,
        num_samples=20
    )

    forecasts = list(forecast_it)
    targets = list(ts_it)

    agg_metric, _ = evaluator(targets, forecasts, num_series=len(test))

    # save metrics
    with open(os.path.join(PREDICTIONS_FOLDER, f"agg_metrics__exchange_{nans_prob}.json"), 'w') as fp:
        json.dump(agg_metric, fp)
    
    # save forecasts
    with open(os.path.join(PREDICTIONS_FOLDER, f"targets__exchange_{nans_prob}.json"), 'wb') as handle:
        pickle.dump(targets, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    with open(os.path.join(PREDICTIONS_FOLDER, f"forecasts__exchange_{nans_prob}.json"), 'wb') as handle:
        pickle.dump(forecasts, handle, protocol=pickle.HIGHEST_PROTOCOL)

    exchange_dataset_test = exchange_test_grouper(exchange.test)


### NaNs probability = 0.0 ###



/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/split.py:181: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  sliced_entry[FieldName.START] += offset
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/transform/feature.py:364: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  index = pd.period_range(start, periods=length, freq=start.freq)
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/transform/feature.py:364: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  index = pd.period_range(start, periods=length, freq=start.freq)
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/transform/split.py:150: FutureWarning: Period with BDay freq 


### NaNs probability = 0.1 ###



/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/util.py:29: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return pd.period_range(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/util.py:29: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  return pd.period_range(
Running evaluation: 0it [00:00, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/model/forecast.py:365: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  self._index = pd.period_range(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/model/forecast.py:365: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeI


### NaNs probability = 0.2 ###



...

/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/util.py:29: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return pd.period_range(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/util.py:29: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  return pd.period_range(
Running evaluation: 0it [00:00, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/model/forecast.py:365: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  self._index = pd.period_range(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/model/forecast.py:365: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeI


### NaNs probability = 0.7000000000000001 ###



/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/util.py:29: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return pd.period_range(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/util.py:29: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  return pd.period_range(
Running evaluation: 0it [00:00, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/model/forecast.py:365: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  self._index = pd.period_range(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/model/forecast.py:365: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeI


### NaNs probability = 0.8 ###



/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/util.py:29: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  return pd.period_range(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/dataset/util.py:29: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  return pd.period_range(
Running evaluation: 0it [00:00, ?it/s]/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/model/forecast.py:365: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  self._index = pd.period_range(
/Users/eliseev_s/anaconda3/envs/env1/lib/python3.9/site-packages/gluonts/model/forecast.py:365: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeI